In [4]:

import numpy as np 
import pandas as pd 
# import tensorflow as tf
import matplotlib.pyplot as plt
import warnings 
from IPython.display import display
import fastparquet
import pyarrow

warnings.filterwarnings('ignore')

In [5]:
path = "/home/biniyam/Downloads/data_kaggle/"

de_train = pd.read_parquet(path + "de_train.parquet")
id_map = pd.read_csv(path + "id_map.csv")
sample_submission = pd.read_csv(path + "sample_submission.csv")

In [6]:
de_train

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,T regulatory cells,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,-0.014372,-0.122464,-0.456366,-0.147894,-0.545382,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
610,NK cells,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-0.455549,0.188181,0.595734,-0.100299,0.786192,...,-1.236905,0.003854,-0.197569,-0.175307,0.101391,1.028394,0.034144,-0.231642,1.023994,-0.064760
611,T cells CD4+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.338168,-0.109079,0.270182,-0.436586,-0.069476,...,0.077579,-1.101637,0.457201,0.535184,-0.198404,-0.005004,0.552810,-0.209077,0.389751,-0.337082
612,T cells CD8+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.101138,-0.409724,-0.606292,-0.071300,-0.001789,...,0.005951,-0.893093,-1.003029,-0.080367,-0.076604,0.024849,0.012862,-0.029684,0.005506,-1.733112


In [7]:
id_map.head()

,id,cell_type,sm_name
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,1,B cells,ABT-199 (GDC-0199)
2,2,B cells,ABT737
3,3,B cells,AMD-070 (hydrochloride)
4,4,B cells,AT 7867


In [8]:
id_map.describe(include='all')

,id,cell_type,sm_name
count,255.000000,255,255
unique,NaN,2,129
top,NaN,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
freq,NaN,128,2
mean,127.000000,NaN,NaN
std,73.756356,NaN,NaN
min,0.000000,NaN,NaN
25%,63.500000,NaN,NaN
50%,127.000000,NaN,NaN
75%,190.500000,NaN,NaN


In [9]:
sample_submission.head()

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
de_train.isnull().sum()

cell_type      0
sm_name        0
sm_lincs_id    0
SMILES         0
control        0
              ..
ZXDB           0
ZXDC           0
ZYG11B         0
ZYX            0
ZZEF1          0
Length: 18216, dtype: int64

In [11]:
unique = de_train['sm_name'].unique()
unique, print({len(unique)})

{146}


(array(['Clotrimazole', 'Mometasone Furoate', 'Idelalisib', 'Vandetanib',
        'Bosutinib', 'Ceritinib', 'Lamivudine', 'Crizotinib',
        'Cabozantinib', 'Flutamide', 'Dasatinib', 'Selumetinib',
        'Trametinib', 'ABT-199 (GDC-0199)', 'Oxybenzone', 'Vorinostat',
        'Raloxifene', 'Linagliptin', 'Lapatinib', 'Canertinib',
        'Disulfiram', 'Vardenafil', 'Palbociclib', 'Ricolinostat',
        'Dabrafenib', 'Proscillaridin A;Proscillaridin-A', 'IN1451',
        'Ixabepilone', 'CEP-18770 (Delanzomib)', 'RG7112', 'MK-5108',
        'Resminostat', 'IMD-0354', 'Alvocidib', 'LY2090314',
        'Methotrexate', 'LDN 193189', 'Tacalcitol', 'Colchicine', 'R428',
        'TL_HRAS26', 'BMS-387032', 'CGP 60474', 'TIE2 Kinase Inhibitor',
        'PD-0325901', 'Isoniazid', 'GSK-1070916', 'Masitinib',
        'Saracatinib', 'CC-401', 'Decitabine', 'Ketoconazole',
        'HYDROXYUREA', 'BAY 61-3606', 'Navitoclax', 'Porcn Inhibitor III',
        'GW843682X', 'Prednisolone', 'Tamatinib'

In [12]:
de_train['cell_type'].unique()

array(['NK cells', 'T cells CD4+', 'T cells CD8+', 'T regulatory cells',
       'B cells', 'Myeloid cells'], dtype=object)

In [13]:
de_train[de_train["control"] == True].count()

cell_type      12
sm_name        12
sm_lincs_id    12
SMILES         12
control        12
               ..
ZXDB           12
ZXDC           12
ZYG11B         12
ZYX            12
ZZEF1          12
Length: 18216, dtype: int64

In [14]:
# shuffle the data
de_train = de_train.sample(frac=1.0, random_state=42)

In [15]:
de_train.shape

(614, 18216)

In [16]:
de_train["control"].value_counts()

False    602
True      12
Name: control, dtype: int64

In [17]:
de_train["SMILES"].value_counts()

Cc1cnc(-c2cnc(NCCNc3ccc(C#N)cn3)nc2-c2ccc(Cl)cc2Cl)[nH]1                   6
CC(C)C[C@H](NC(=O)CNC(=O)c1cc(Cl)ccc1Cl)B(O)O                              6
O=C(O)c1ccc2cc(-c3ccc(O)c(C45CC6CC(CC(C6)C4)C5)c3)ccc2c1                   6
c1ccc2c(-c3cnn4cc(-c5ccc(N6CCNCC6)cc5)cnc34)ccnc2c1                        6
OC1(c2ccc(Cl)c(C(F)(F)F)c2)CCN(CCCC(c2ccc(F)cc2)c2ccc(F)cc2)CC1            6
                                                                          ..
CC1=NN(c2ccc(C(=O)O)cc2)C(=O)/C1=C/c1ccc(-c2cc(C)c(C)cc2[N+](=O)[O-])o1    4
COc1cc(-c2cc(-c3cccc(O)c3)cnc2N)cc(OC)c1OC                                 4
Nc1ncn([C@H]2C[C@H](O)[C@@H](CO)O2)c(=O)n1                                 4
CN(C)Cc1ccc(S(=O)(=O)n2ccc(/C=C/C(=O)NO)c2)cc1                             3
OCCCNc1cc(-c2ccnc(Nc3cccc(Cl)c3)n2)ccn1                                    3
Name: SMILES, Length: 146, dtype: int64

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
d_copy = de_train[['sm_name', 'SMILES']]
d_copy = d_copy.sort_values("sm_name")
display(d_copy)


d_copy['SMILE_MODE'] = d_copy.groupby('sm_name')['SMILES'].transform(lambda x: x.mode().iloc[0])
d_copy[d_copy.SMILE_MODE != d_copy.SMILES].shape


,sm_name,SMILES
592,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C
590,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C
593,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C
591,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C
56,ABT-199 (GDC-0199),CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...
...,...,...
66,Vorinostat,O=C(CCCCCCC(=O)Nc1ccccc1)NO
523,YK 4-279,COc1ccc(C(=O)CC2(O)C(=O)Nc3c(Cl)ccc(Cl)c32)cc1
524,YK 4-279,COc1ccc(C(=O)CC2(O)C(=O)Nc3c(Cl)ccc(Cl)c32)cc1
526,YK 4-279,COc1ccc(C(=O)CC2(O)C(=O)Nc3c(Cl)ccc(Cl)c32)cc1


(0, 3)

In [19]:
id_map.sm_name.isin(de_train.sm_name.unique()).sum(), id_map.shape[0]

(255, 255)

In [24]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, AllChem


# Function to compute features
def compute_features(smiles_str):
    mol = Chem.MolFromSmiles(smiles_str)
    if mol is None:
        return [None]*24
    
    return [
        Descriptors.MolWt(mol),
        mol.GetNumAtoms(),
        mol.GetNumBonds(),
        Lipinski.NumHDonors(mol),
        Lipinski.NumHAcceptors(mol),
        Descriptors.NumAromaticRings(mol),
        Descriptors.NumSaturatedRings(mol),
        Descriptors.NumAliphaticRings(mol),
        Descriptors.NumAromaticCarbocycles(mol),
        Descriptors.NumSaturatedCarbocycles(mol),
        Descriptors.NumAliphaticCarbocycles(mol),
        Descriptors.NumAromaticHeterocycles(mol),
        Descriptors.NumSaturatedHeterocycles(mol),
        Descriptors.NumAliphaticHeterocycles(mol),
#         Chem.GetSSSR(mol),
        AllChem.CalcNumRotatableBonds(mol),
        Descriptors.TPSA(mol),
        Descriptors.NOCount(mol),
        Descriptors.NumHeteroatoms(mol),
        Descriptors.NumValenceElectrons(mol),
        Descriptors.MaxPartialCharge(mol),
        Descriptors.MinPartialCharge(mol),
        Descriptors.BCUT2D_MWHI(mol),
        Descriptors.BCUT2D_MWLOW(mol)
    ]

# Apply the function to the SMILES column
unique_smiles = de_train[['sm_name', 'SMILES']].drop_duplicates().reset_index(drop=True)
features = unique_smiles['SMILES'].apply(compute_features)

# Convert the result to a DataFrame
features_df = pd.DataFrame(features.tolist(), columns=[
    'Molecular_Weight', 'Num_Atoms', 'Num_Bonds', 
    'Num_H_Donors', 'Num_H_Acceptors', 'Num_Aromatic_Rings',
    'Num_Saturated_Rings', 'Num_Aliphatic_Rings',
    'Num_Aromatic_Carbocycles', 'Num_Saturated_Carbocycles',
    'Num_Aliphatic_Carbocycles', 'Num_Aromatic_Heterocycles',
    'Num_Saturated_Heterocycles', 'Num_Aliphatic_Heterocycles',
#     'SSSR', 
    'Num_Rotatable_Bonds', 'TPSA', 'NO_Count',
    'Num_Heteroatoms', 'Num_Valence_Electrons',
    'Max_Partial_Charge', 'Min_Partial_Charge',
    'BCUT2D_MWHI', 'BCUT2D_MWLOW'
])


fts_df = pd.concat([unique_smiles, features_df], axis=1)
display(fts_df.head(10))
display(fts_df.isna().sum())
fts_df.dtypes

,sm_name,SMILES,Molecular_Weight,Num_Atoms,Num_Bonds,Num_H_Donors,Num_H_Acceptors,Num_Aromatic_Rings,Num_Saturated_Rings,Num_Aliphatic_Rings,...,Num_Aliphatic_Heterocycles,Num_Rotatable_Bonds,TPSA,NO_Count,Num_Heteroatoms,Num_Valence_Electrons,Max_Partial_Charge,Min_Partial_Charge,BCUT2D_MWHI,BCUT2D_MWLOW
0,TPCA-1,NC(=O)Nc1sc(-c2ccc(F)cc2)cc1C(N)=O,279.296,19,20,3,3,2,0,0,...,0,3,98.21,5,7,98,0.316752,-0.365505,32.133570,10.140439
1,Dovitinib,CN1CCN(c2ccc3[nH]c(-c4c(N)c5c(F)cccc5[nH]c4=O)...,392.438,29,33,3,5,4,1,1,...,1,2,94.04,7,8,148,0.261083,-0.397332,19.142180,10.089609
2,Tacalcitol,C=C1/C(=C\C=C2/CCC[C@]3(C)[C@@H]([C@H](C)CC[C@...,416.646,30,32,3,3,0,3,3,...,0,6,60.69,3,3,170,0.081130,-0.392864,16.284635,9.604044
3,Atorvastatin,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,558.650,41,44,4,5,4,0,0,...,0,12,111.79,7,8,214,0.305472,-0.481159,19.142149,9.941042
4,MK-5108,O=C(O)[C@]1(Cc2cccc(Nc3nccs3)n2)CC[C@@H](Oc2cc...,461.946,31,34,2,6,3,1,1,...,0,7,84.34,6,9,162,0.309532,-0.487303,35.495693,9.699819
5,Imatinib,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,493.615,37,41,2,7,4,1,1,...,1,7,86.28,8,8,188,0.255159,-0.323844,16.149175,10.098650
6,Foretinib,COc1cc2c(Oc3ccc(NC(=O)C4(C(=O)Nc5ccc(F)cc5)CC4...,632.664,46,51,2,8,4,2,2,...,1,12,111.25,10,12,240,0.239586,-0.492842,19.143603,10.045556
7,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,344.845,25,28,0,2,4,0,0,...,0,4,17.82,2,3,122,0.122586,-0.318903,35.495692,9.764476
8,Tivantinib,O=C1NC(=O)[C@@H](c2cn3c4c(cccc24)CCC3)[C@@H]1c...,369.424,28,33,2,3,4,1,2,...,2,2,66.89,5,5,138,0.234847,-0.360863,16.174357,9.825496
9,Linagliptin,CC#CCn1c(N2CCC[C@@H](N)C2)nc2c1c(=O)n(Cc1nc(C)...,472.553,35,39,1,10,4,1,1,...,1,4,116.86,10,10,180,0.332402,-0.340600,16.161840,10.079917


sm_name                       0
SMILES                        0
Molecular_Weight              0
Num_Atoms                     0
Num_Bonds                     0
Num_H_Donors                  0
Num_H_Acceptors               0
Num_Aromatic_Rings            0
Num_Saturated_Rings           0
Num_Aliphatic_Rings           0
Num_Aromatic_Carbocycles      0
Num_Saturated_Carbocycles     0
Num_Aliphatic_Carbocycles     0
Num_Aromatic_Heterocycles     0
Num_Saturated_Heterocycles    0
Num_Aliphatic_Heterocycles    0
Num_Rotatable_Bonds           0
TPSA                          0
NO_Count                      0
Num_Heteroatoms               0
Num_Valence_Electrons         0
Max_Partial_Charge            0
Min_Partial_Charge            0
BCUT2D_MWHI                   0
BCUT2D_MWLOW                  0
dtype: int64

sm_name                        object
SMILES                         object
Molecular_Weight              float64
Num_Atoms                       int64
Num_Bonds                       int64
Num_H_Donors                    int64
Num_H_Acceptors                 int64
Num_Aromatic_Rings              int64
Num_Saturated_Rings             int64
Num_Aliphatic_Rings             int64
Num_Aromatic_Carbocycles        int64
Num_Saturated_Carbocycles       int64
Num_Aliphatic_Carbocycles       int64
Num_Aromatic_Heterocycles       int64
Num_Saturated_Heterocycles      int64
Num_Aliphatic_Heterocycles      int64
Num_Rotatable_Bonds             int64
TPSA                          float64
NO_Count                        int64
Num_Heteroatoms                 int64
Num_Valence_Electrons           int64
Max_Partial_Charge            float64
Min_Partial_Charge            float64
BCUT2D_MWHI                   float64
BCUT2D_MWLOW                  float64
dtype: object

In [25]:
# Create features and and labels for reverse model 18211 features and 152 labels for true model
features_columns = ["cell_type", "sm_name"]
labels_columns=["cell_type","sm_name","sm_lincs_id","SMILES","control"]
labels = de_train.drop(columns=labels_columns)
L_cols = list(labels.columns)
features = pd.DataFrame(de_train, columns=features_columns+L_cols)

In [26]:
features = features.merge(fts_df, on='sm_name').drop(columns=['SMILES'])
features

,cell_type,sm_name,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,...,Num_Aliphatic_Heterocycles,Num_Rotatable_Bonds,TPSA,NO_Count,Num_Heteroatoms,Num_Valence_Electrons,Max_Partial_Charge,Min_Partial_Charge,BCUT2D_MWHI,BCUT2D_MWLOW
0,T regulatory cells,TPCA-1,2.175274,1.219289,2.396227,2.885988,3.384956,3.267348,0.134622,1.829941,...,0,3,98.21,5,7,98,0.316752,-0.365505,32.133570,10.140439
1,T cells CD8+,TPCA-1,0.408349,-0.919647,0.173442,-0.194472,0.340177,-0.730095,-0.898621,-0.978824,...,0,3,98.21,5,7,98,0.316752,-0.365505,32.133570,10.140439
2,T cells CD4+,TPCA-1,0.031096,-0.247004,0.193300,0.940602,0.700820,0.113228,0.081981,-0.157682,...,0,3,98.21,5,7,98,0.316752,-0.365505,32.133570,10.140439
3,NK cells,TPCA-1,-0.803042,-0.164946,-0.357530,0.106471,2.152284,0.555610,0.127219,0.110150,...,0,3,98.21,5,7,98,0.316752,-0.365505,32.133570,10.140439
4,NK cells,Dovitinib,0.214531,-0.115171,0.326515,0.032289,0.220971,-0.463554,1.016849,0.153039,...,1,2,94.04,7,8,148,0.261083,-0.397332,19.142180,10.089609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,T cells CD8+,PF-04691502,1.795443,0.189875,1.716503,2.459436,0.057301,0.356322,0.720463,0.142335,...,0,6,125.38,9,9,164,0.259905,-0.480984,16.495391,9.917773
610,NK cells,Decitabine,0.850528,0.160443,0.621358,-0.136017,-0.107275,1.028186,0.341713,1.359028,...,1,2,123.49,8,8,88,0.353679,-0.393569,16.542721,10.171001
611,T cells CD8+,Decitabine,1.205679,0.328051,-0.273971,-0.022163,-0.057408,-0.024851,0.086890,-0.499899,...,1,2,123.49,8,8,88,0.353679,-0.393569,16.542721,10.171001
612,T regulatory cells,Decitabine,0.752723,0.849754,1.033994,2.321358,1.213635,1.152229,0.428171,0.064429,...,1,2,123.49,8,8,88,0.353679,-0.393569,16.542721,10.171001


In [27]:
labels

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
350,2.175274,1.219289,2.396227,2.885988,3.384956,3.267348,0.134622,1.829941,-0.956191,-0.404039,...,-1.062199,1.178715,0.180416,0.755145,1.809688,1.644712,0.701406,0.137341,0.230949,0.179458
377,0.214531,-0.115171,0.326515,0.032289,0.220971,-0.463554,1.016849,0.153039,0.292684,0.038020,...,0.438939,0.216506,-0.108654,-0.063070,0.006721,-0.593161,-0.044022,0.475729,0.356555,-0.038020
163,0.391257,-0.052922,0.540292,0.718991,1.083183,1.021599,-0.097250,0.306833,-3.331719,0.235092,...,-1.755527,0.556352,0.089976,-0.000729,0.419802,1.020842,0.367313,0.486345,-0.856723,0.191009
609,-0.014372,-0.122464,-0.456366,-0.147894,-0.545382,-0.544709,0.282458,-0.431359,-0.364961,0.043123,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
132,0.094873,1.067610,-0.745980,1.083842,0.638504,0.307649,-0.038108,0.772186,-0.736735,-0.462323,...,0.279780,0.489431,0.147217,0.292313,0.029266,0.330628,1.038142,-0.194884,1.078132,0.057001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,1.080779,0.226991,2.440729,2.581894,1.773980,1.989227,-0.074095,0.906555,2.214564,-0.182784,...,-0.276043,0.768273,1.351487,2.359651,1.470219,0.866639,0.467307,0.663700,-0.122574,-0.315800
106,-0.161665,0.269009,-1.770091,-0.198444,0.954753,1.155912,-0.312552,0.621897,-0.154055,0.331780,...,0.131425,-1.112697,-0.589313,-0.660929,0.016530,0.319136,0.299521,-0.383452,0.267828,-1.100267
270,-0.304126,0.443651,0.688265,-0.101457,-0.005912,-0.004914,-0.247434,0.281614,-0.915735,-0.032908,...,-0.855096,-0.499096,-0.211083,-0.495621,-0.233734,1.120530,-0.109096,0.128084,0.313468,-0.512444
435,0.482294,-1.262924,-0.545405,-0.497739,-0.750201,0.959704,0.527966,1.198200,-0.482740,0.635863,...,0.444958,0.029279,0.337660,0.025025,0.035943,0.691171,0.352091,0.046012,-0.760812,-0.168719


In [28]:
# Get test data 
test_data = pd.DataFrame(id_map, columns=features_columns)
test_data = test_data.merge(fts_df, on='sm_name').drop(columns=['SMILES'])

In [29]:
test_data

,cell_type,sm_name,Molecular_Weight,Num_Atoms,Num_Bonds,Num_H_Donors,Num_H_Acceptors,Num_Aromatic_Rings,Num_Saturated_Rings,Num_Aliphatic_Rings,...,Num_Aliphatic_Heterocycles,Num_Rotatable_Bonds,TPSA,NO_Count,Num_Heteroatoms,Num_Valence_Electrons,Max_Partial_Charge,Min_Partial_Charge,BCUT2D_MWHI,BCUT2D_MWLOW
0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,354.418,26,29,1,9,3,1,1,...,1,3,107.87,9,9,136,0.227706,-0.377774,16.477573,10.189388
1,Myeloid cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,354.418,26,29,1,9,3,1,1,...,1,3,107.87,9,9,136,0.227706,-0.377774,16.477573,10.189388
2,B cells,ABT-199 (GDC-0199),868.457,61,68,3,11,5,2,3,...,2,13,172.03,14,16,320,0.293190,-0.454803,35.495694,9.722170
3,Myeloid cells,ABT-199 (GDC-0199),868.457,61,68,3,11,5,2,3,...,2,13,172.03,14,16,320,0.293190,-0.454803,35.495694,9.722170
4,B cells,ABT737,813.446,56,61,2,10,5,1,1,...,1,16,128.13,11,14,292,0.293198,-0.375866,35.495694,9.989707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,B cells,Vorinostat,264.325,19,19,3,3,1,0,0,...,0,8,78.43,5,5,104,0.242829,-0.326098,16.466913,10.105131
251,Myeloid cells,Vorinostat,264.325,19,19,3,3,1,0,0,...,0,8,78.43,5,5,104,0.242829,-0.326098,16.466913,10.105131
252,B cells,YK 4-279,366.200,24,26,2,4,2,0,1,...,1,4,75.63,5,7,124,0.261158,-0.496768,35.496788,9.877523
253,Myeloid cells,YK 4-279,366.200,24,26,2,4,2,0,1,...,1,4,75.63,5,7,124,0.261158,-0.496768,35.496788,9.877523


# Preprocess Data

In [30]:
cat_features = ['cell_type', 'sm_name']

train_test = pd.concat([features.assign(train=1), test_data.assign(train=0)])
train_test = pd.get_dummies(train_test, cat_features)
train_test

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,sm_name_Tivozanib,sm_name_Topotecan,sm_name_Tosedostat,sm_name_Trametinib,sm_name_UNII-BXU45ZH6LI,sm_name_Vandetanib,sm_name_Vanoxerine,sm_name_Vardenafil,sm_name_Vorinostat,sm_name_YK 4-279
0,2.175274,1.219289,2.396227,2.885988,3.384956,3.267348,0.134622,1.829941,-0.956191,-0.404039,...,0,0,0,0,0,0,0,0,0,0
1,0.408349,-0.919647,0.173442,-0.194472,0.340177,-0.730095,-0.898621,-0.978824,-0.351777,-0.153040,...,0,0,0,0,0,0,0,0,0,0
2,0.031096,-0.247004,0.193300,0.940602,0.700820,0.113228,0.081981,-0.157682,0.727687,0.427128,...,0,0,0,0,0,0,0,0,0,0
3,-0.803042,-0.164946,-0.357530,0.106471,2.152284,0.555610,0.127219,0.110150,0.332709,0.116285,...,0,0,0,0,0,0,0,0,0,0
4,0.214531,-0.115171,0.326515,0.032289,0.220971,-0.463554,1.016849,0.153039,0.292684,0.038020,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1


In [31]:
bool_cols = train_test.select_dtypes(bool).columns

train_test[bool_cols] = train_test[bool_cols].astype(int)
train_test

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,sm_name_Tivozanib,sm_name_Topotecan,sm_name_Tosedostat,sm_name_Trametinib,sm_name_UNII-BXU45ZH6LI,sm_name_Vandetanib,sm_name_Vanoxerine,sm_name_Vardenafil,sm_name_Vorinostat,sm_name_YK 4-279
0,2.175274,1.219289,2.396227,2.885988,3.384956,3.267348,0.134622,1.829941,-0.956191,-0.404039,...,0,0,0,0,0,0,0,0,0,0
1,0.408349,-0.919647,0.173442,-0.194472,0.340177,-0.730095,-0.898621,-0.978824,-0.351777,-0.153040,...,0,0,0,0,0,0,0,0,0,0
2,0.031096,-0.247004,0.193300,0.940602,0.700820,0.113228,0.081981,-0.157682,0.727687,0.427128,...,0,0,0,0,0,0,0,0,0,0
3,-0.803042,-0.164946,-0.357530,0.106471,2.152284,0.555610,0.127219,0.110150,0.332709,0.116285,...,0,0,0,0,0,0,0,0,0,0
4,0.214531,-0.115171,0.326515,0.032289,0.220971,-0.463554,1.016849,0.153039,0.292684,0.038020,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1


In [32]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

num_cols = train_test.select_dtypes("float64").drop(columns=L_cols).columns 
train_test[num_cols] = scaler.fit_transform(train_test[num_cols])

In [33]:
filtered_features = train_test[train_test.columns.drop(L_cols)]
correlation_matrix = filtered_features.corr()

columns_to_drop = set()  
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.75:  
            colname = correlation_matrix.columns[i]
            columns_to_drop.add(colname)
            
columns_to_drop

{'NO_Count',
 'Num_Aliphatic_Carbocycles',
 'Num_Aliphatic_Heterocycles',
 'Num_Aliphatic_Rings',
 'Num_Atoms',
 'Num_Bonds',
 'Num_Heteroatoms',
 'Num_Valence_Electrons',
 'TPSA'}

In [34]:
drop_cols = list(columns_to_drop)
features = train_test[train_test.train == 1].drop(columns=['train'] + drop_cols)
test_data = train_test[train_test.train == 0].drop(columns=['train'] + L_cols + drop_cols)

features.shape, test_data.shape

((614, 18377), (255, 166))

# SPLIT 

In [35]:
newly_created_fts = [i for i in features.columns.drop(L_cols) if 'sm_name_' not in i and 'cell_type_' not in i]
train_test[newly_created_fts].describe()

,Molecular_Weight,Num_H_Donors,Num_H_Acceptors,Num_Aromatic_Rings,Num_Saturated_Rings,Num_Aromatic_Carbocycles,Num_Saturated_Carbocycles,Num_Aromatic_Heterocycles,Num_Saturated_Heterocycles,Num_Rotatable_Bonds,Max_Partial_Charge,Min_Partial_Charge,BCUT2D_MWHI,BCUT2D_MWLOW
count,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000
mean,0.403471,1.895282,5.995397,3.004603,0.756041,1.628308,0.248562,1.376295,0.507480,5.811277,0.456020,0.575777,0.129149,0.328944
std,0.132993,1.233869,2.283476,1.312814,0.890478,0.981947,0.680802,1.011428,0.600692,2.903408,0.172104,0.213808,0.194297,0.112703
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.320945,1.000000,4.000000,2.000000,0.000000,1.000000,0.000000,1.000000,0.000000,4.000000,0.385184,0.364401,0.015007,0.278718
50%,0.408862,2.000000,6.000000,3.000000,1.000000,2.000000,0.000000,1.000000,0.000000,6.000000,0.447161,0.637902,0.041207,0.345710
75%,0.473480,3.000000,7.000000,4.000000,1.000000,2.000000,0.000000,2.000000,1.000000,7.000000,0.561257,0.752992,0.184646,0.384091
max,1.000000,6.000000,12.000000,6.000000,4.000000,5.000000,4.000000,4.000000,2.000000,16.000000,1.000000,1.000000,1.000000,1.000000


In [36]:
training_fts = features.columns.drop(L_cols)

full_features = features[training_fts].astype("float32").values
full_labels = features[L_cols].astype("float32").values
test_features = test_data[training_fts].astype("float32").values

print("full_features shape: ", full_features.shape)
print("full labels shape: ", full_labels.shape)
print("test features shape: ", test_features.shape)

full_features shape:  (614, 166)
full labels shape:  (614, 18211)
test features shape:  (255, 166)


Now predict!

In [43]:
from sklearn.multioutput import MultiOutputRegressor
from catboost import CatBoostRegressor


def get_model():
    estimator = CatBoostRegressor(verbose=1001, task_type='GPU')
    model = MultiOutputRegressor(estimator=estimator, n_jobs=-1)

    return model

In [45]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# Define the number of folds (K)
num_folds = 10

# Initialize lists to store the model's performance scores
mae_scores = []
mrrmse_scores = []
predictions = []

# Initialize the KFold object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=51)

# Loop through the K folds
for inx, (train_index, val_index) in enumerate(kf.split(features)):
    print("EPOCH   {inx + 1}\n" + "-"*50)
    # Convert indices to integers and split the data
    train_index = train_index.astype(int)
    val_index = val_index.astype(int)
    X_train_, X_val_ = full_features[train_index], full_features[val_index]
    y_train_, y_val_ = full_labels[train_index], full_labels[val_index]

    # Train the model on X_train and y_train
    model = get_model()
    model.fit(X_train_, y_train_)

    # Make predictions on the validation set
    y_preds = model.predict(X_val_)
    preds = model.predict(test_features, batch_size=1)
    predictions.append(preds)
    
    
    # Calculate the Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_val_, y_preds)
    mae_scores.append(mae)

    # Calculate the Mean Rowwise Root Mean Square Error (MRRMSE)
    rowwise_rmse = np.sqrt(np.mean(np.square(y_val_ - y_preds), axis=1))
    mrrmse_score = np.mean(rowwise_rmse)
    mrrmse_scores.append(mrrmse_score)

# Calculate the mean and standard deviation of MAE and MRRMSE scores
mean_mae = np.mean(mae_scores)
mean_mrrmse = np.mean(mrrmse_scores)

# Print the results
print(f'Average MAE across {num_folds} folds: {mean_mae:.4f} ')
print(f'Average MRRMSE across {num_folds} folds: {mean_mrrmse:.4f}')

EPOCH   {inx + 1}
--------------------------------------------------


Application terminated with error: ??+0 (0x7F2BABB7FA92)
??+0 (0x7F2BABB7F8D4)
??+0 (0x7F2BABB79E92)
??+0 (0x7F2BABB78BEF)
??+0 (0x7F2BABB7A518)
??+0 (0x7F2BAA7F5AF7)
??+0 (0x7F2BAA7F59A7)
??+0 (0x7F2BC3694AC3)
??+0 (0x7F2BC3726A40)

(TCatBoostException) /src/catboost/catboost/cuda/cuda_lib/cuda_base.h:183: CUDA error 2: out of memory
uncaught exception:
    address -> 0x540c5590410
    what() -> "/src/catboost/catboost/cuda/cuda_lib/cuda_base.h:183: CUDA error 2: out of memory"
    type -> TCatBoostException
Application terminated with error: ??+0 (0x7F774BB7FA92)
??+0 (0x7F774BB7F8D4)
??+0 (0x7F774BB79E92)
??+0 (0x7F774BB78BEF)
??+0 (0x7F774BB7A518)
??+0 (0x7F774A7F5AF7)
??+0 (0x7F774A7F59A7)
??+0 (0x7F7762094AC3)
??+0 (0x7F7762126A40)

(TCatBoostException) /src/catboost/catboost/cuda/cuda_lib/cuda_base.h:183: CUDA error 2: out of memory
uncaught exception:
    address -> 0x2e921590410
    what() -> "/src/catboost/catboost/cuda/cuda_lib/cuda_base.h:183: CUDA error 2: out of memory"
 

Learning rate set to 0.037369
Learning rate set to 0.037369
Learning rate set to 0.037369
0:	learn: 1.2032298	total: 39.7ms	remaining: 39.7s
0:	learn: 0.7200767	total: 28.9ms	remaining: 28.9s


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGABRT(-6)}

- Average MAE across 10 folds: 0.7755 
- Average MRRMSE across 10 folds: 1.1208

In [ ]:
preds = np.mean(predictions, axis=0)

In [123]:
sample_columns = sample_submission.columns
sample_columns= sample_columns[1:]
submission_df = pd.DataFrame(preds, columns=sample_columns)

In [124]:
submission_df.insert(0, 'id', range(255))

In [125]:
sample_submission

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251,251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252,252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253,253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
submission_df

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,-0.031284,0.037411,0.348645,0.525145,0.722608,0.226565,-0.120076,-0.289897,0.089700,...,-1.423311,0.055185,-0.242945,0.138092,-0.022121,-0.024660,0.517806,0.091833,-0.529569,0.552286
1,1,0.522643,-0.124787,1.378281,1.117512,2.854251,2.268515,0.164498,-0.042300,-0.995515,...,-1.610866,0.025155,-0.154674,-1.340841,1.142312,-0.221657,0.091149,0.408186,-0.222756,1.238389
2,2,0.242976,0.251904,0.524342,0.269953,1.214070,0.998073,-0.305648,0.164700,0.541683,...,-0.391441,0.178175,-0.553115,0.285951,0.418873,0.336454,0.686956,-0.049785,-0.933905,0.106384
3,3,0.030152,0.096338,0.068814,0.006803,0.121986,0.118194,0.038470,0.087553,0.033149,...,0.013035,-0.035373,-0.160701,0.115392,0.086651,0.141281,0.045848,0.009034,-0.114532,-0.109481
4,4,0.644170,0.016944,0.101660,0.763080,2.339300,1.315502,-0.247679,0.461496,0.671314,...,-0.275629,-0.004896,-0.009710,0.994432,0.517386,0.570500,0.723148,0.118695,-0.656513,0.134547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,0.610164,-0.267190,0.647085,1.823325,1.708368,0.371159,-0.179656,-0.429344,0.437646,...,-0.350401,0.052098,-0.278275,0.127273,-0.360866,0.054366,0.300606,0.365628,0.460470,0.300960
251,251,0.650234,0.246085,1.328946,1.757787,2.802367,1.794192,0.315409,-0.161016,-0.058250,...,-1.210996,-0.031632,-0.396195,-0.507855,0.811441,0.031790,-0.687351,0.315662,0.430685,0.381567
252,252,0.042919,-0.021228,0.342843,0.228963,0.597797,0.412636,-0.220218,0.031403,0.246423,...,-0.385101,0.110201,-0.427260,-0.156606,0.138463,0.095814,0.546707,-0.125375,-0.508994,0.114963
253,253,0.155251,-0.025122,0.215164,-0.051778,0.350229,0.210370,0.168570,0.096882,0.043617,...,-0.003584,-0.045957,-0.248022,-0.035394,0.065220,0.187420,-0.110221,-0.010135,-0.265545,-0.177669


In [127]:
submission_df.to_csv("submission.csv", index=False)

In [128]:
!zip submission_preds.zip /kaggle/working/submission.csv

  adding: kaggle/working/submission.csv (deflated 58%)
